In [1]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import string

# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
import pickle
import argparse
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

user_dict = pickle.load(open('./user_id_dict.pkl'))
item_dict = pickle.load(open('./item_id_dict.pkl'))
item_genre_dict = pickle.load(open('./item_genre_dict.pkl'))
genre_count = pickle.load(open('./genre_count.pkl'))

user_idd_list = pickle.load(open('./user_idd_list.pkl'))
item_idd_list = pickle.load(open('./item_idd_list.pkl'))
key_genre = pickle.load(open('./key_genre.pkl'))
item_idd_genre_list = pickle.load(open('./item_idd_genre_list.pkl'))

train_df = pickle.load(open('./training_df.pkl'))

In [3]:
'''HFT train set preprocessing'''

from time import *
from keras.preprocessing.text import text_to_word_sequence
from nltk.stem.porter import PorterStemmer

user_item_id_tuple_list = []
for pair in list(train_df.loc[:, ['user_id', 'item_id']].values):
    user_item_id_tuple_list.append(tuple(pair))
user_item_id_tuple_set = set(user_item_id_tuple_list)

ps = PorterStemmer(PorterStemmer.NLTK_EXTENSIONS)
stop_words = set(stopwords.words('english'))

f = open('./review.json')
reviews = f.readlines()
f.close()

fw = open('./HFT_Yelp.txt', 'w')

user_set = set(user_dict.keys())
item_set = set(item_dict.keys())

l_r = len(reviews)
l = 0
for review in reviews:
    review = json.loads(review)
    user = review['user_id']
    item = review['business_id']
    if l % 100000 == 0:
        print(str(l) + '/' + str(l_r))
    l += 1
    if not (user in user_set and item in item_set):
        continue
    u_idx = user_dict[user]
    i_idx = item_dict[item]
    if tuple([u_idx, i_idx]) in user_item_id_tuple_set:
        rating = review['stars']
        temp = (review['text'])
        tokens = text_to_word_sequence(temp, filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
        tokens[:] = (ps.stem(t) for t in tokens if t != '' and not t.isdigit() and t not in stop_words)
        temp = ' '.join(tokens)
        temp = filter(lambda x: x in string.printable, temp)
        wcount = len(temp.split())
        fw.write(user + ' ' + item + ' ' + str(rating) + ' 0 ' + str(wcount) + ' ' + temp + '\n')
    
fw.close()

Using TensorFlow backend.


0/5261669
100000/5261669
200000/5261669
300000/5261669
400000/5261669
500000/5261669
600000/5261669
700000/5261669
800000/5261669
900000/5261669
1000000/5261669
1100000/5261669
1200000/5261669
1300000/5261669
1400000/5261669
1500000/5261669
1600000/5261669
1700000/5261669
1800000/5261669
1900000/5261669
2000000/5261669
2100000/5261669
2200000/5261669
2300000/5261669
2400000/5261669
2500000/5261669
2600000/5261669
2700000/5261669
2800000/5261669
2900000/5261669
3000000/5261669
3100000/5261669
3200000/5261669
3300000/5261669
3400000/5261669
3500000/5261669
3600000/5261669
3700000/5261669
3800000/5261669
3900000/5261669
4000000/5261669
4100000/5261669
4200000/5261669
4300000/5261669
4400000/5261669
4500000/5261669
4600000/5261669
4700000/5261669
4800000/5261669
4900000/5261669
5000000/5261669
5100000/5261669
5200000/5261669


In [4]:
'''HFT test and vali set preprocessing'''
user_dict = pickle.load(open('./user_id_dict.pkl'))
item_dict = pickle.load(open('./item_id_dict.pkl'))
user_idd_list = pickle.load(open('./user_idd_list.pkl'))
item_idd_list = pickle.load(open('./item_idd_list.pkl'))
test_df = pickle.load(open('./testing_df.pkl'))
vali_df = pickle.load(open('./valiing_df.pkl'))
fw_test = open('./HFT_Yelp_test.txt', 'w')
fw_vali = open('./HFT_Yelp_vali.txt', 'w')

user_set = set(user_dict.keys())
item_set = set(item_dict.keys())

for i in range(len(test_df)):
    user = user_idd_list[test_df.at[i, 'user_id']]
    item = item_idd_list[test_df.at[i, 'item_id']]
    rating = test_df.at[i, 'rating']
    fw_test.write(user + ' ' + item + ' ' + str(rating) + ' 0 0 ' + '\n')
    
for i in range(len(vali_df)):
    user = user_idd_list[vali_df.at[i, 'user_id']]
    item = item_idd_list[vali_df.at[i, 'item_id']]
    rating = vali_df.at[i, 'rating']
    fw_vali.write(user + ' ' + item + ' ' + str(rating) + ' 0 0 ' + '\n')

fw_test.close()
fw_vali.close()